<a href="https://colab.research.google.com/github/suwatoh/Python-learning/blob/main/132_Web%E3%82%A2%E3%83%97%E3%83%AA%E3%82%B1%E3%83%BC%E3%82%B7%E3%83%A7%E3%83%B3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Web アプリケーション
====================

HTTP サーバー
-------------

`http.server` モジュールは、簡易な Web サーバー（HTTP サーバー）を実装するためのクラスを提供している。

``` python
http.server.HTTPServer(server_address, RequestHandlerClass)
```

| コンストラクタ引数 | 意味 |
|:---|:---|
| `server_address` | サーバーの URL を表す文字列とポート番号の 2 要素タプル `(server, address)` を指定する。`server` が空文字ならループバック<br /><br />アドレスを指定したものとみなされる |
| `RequestHandlerClass` | サーバーに到着したリクエストを処理するためのクラスを指定する |

`http.server.HTTPServer` は、クライアントから送られたリクエストの処理を `RequestHandlerClass` に委譲する。 `RequestHandlerClass` とするクラスは以下の条件を満たす必要がある。

  1. イニシャライザ `__init__()` は、3 つの引数 `request`, `client_address`, `server` を受け付ける。
  2. イニシャライザ `__init__()` は、自らあるいは他の関数に委譲する形でリクエストの処理を実装している。

`http.server.HTTPServer` クラスは `socketserver` モジュールの上に構築されている。クラスの継承関係は、下図のとおり。

![](https://www.plantuml.com/plantuml/png/RP0z3i8m38NtdC8Z3Eq962fO697e7bMfIoD2Wk9NG8BUdH9-0XARtnTzVf4hGHCn79not84GADfT3sfjP0CRl3Ak4I49PiyGZYUEvR8HtgQHheeeKTs5o1EPpH-a7z26iq_XAMdZWd02MhF7HFGxNInI6F-LrQjDxlfTbeqUE9OlOLtNsHZLrsSNjwAWppwLfloGFLMnRwURt040)

`http.server.HTTPServer` は、（他の Python ネットワークサーバーと同様に）コンテキストマネージャーとして使用できる。主なメソッドは次のとおり:

| メソッド | 機能 |
|:---|:---|
| `*Server.serve_forever(poll_interval=0.5)` | `shutdown()` が呼ばれるまでクライアントからのリクエストを処理し続ける。具体的には<br /><br /> `RequestHandlerClass(request, client_address, self)` を呼び出すループを実行する。<br /><br />`poll_interval` 秒ごとにシャットダウンをポーリング（定期的な問い合わせ）をする |
| `*Server.shutdown()` | `serve_forever()` ループを停止するように要求し、停止するまで待機する |
| `*Server.server_close()` | サーバーをクリーンアップする |
| `*Server.__enter__()` | 自身を返す |
| `*Server.__exit__(self, *args)` | `server_close()` を呼び出す |

`RequestHandlerClass` の実装は、シンプルな処理でよければ、 `http.server.SimpleHTTPRequestHandler` が使える。このクラスは、 `http.server.BaseHTTPRequestHandler` クラスのサブクラスである。その `__init__()` が呼ばれると、 `GET` リクエストを処理するために `do_GET()` メソッドを呼び出す。 `do_GET()` は、リクエストを現在の作業ディレクトリからの相対的なパスとして解釈することで、リクエストをローカルシステム上のファイルと対応付ける。リクエストがディレクトリに対応付けられた場合、 `index.html` または `index.htm` を（この順序で）チェックする。もしファイルを発見できればその内容を、そうでなければディレクトリ一覧を返す。

`simpleserver.py`:

``` python
from http.server import HTTPServer, SimpleHTTPRequestHandler
server_address = ("", 8000)
with HTTPServer(server_address, SimpleHTTPRequestHandler) as httpd:
    httpd.serve_forever()
```

`index.html`:

``` html
<html lang="ja">
<head><meta charset="UTF-8"></head>
<body><h1>Hello HTTP!</h1></body>
</html>
```

作業ディレクトリに上記の 2 つのファイルを作成し、コマンドラインで `python simpleserver.py` を実行すると、 Web サーバーが立ち上がる。この状態で、Web ブラウザを使って `http://127.0.0.1:8000/` という URLにアクセスすると、 `index.html` の内容が表示される。

<img src="data:image/png;base64,
...
rkJggg==" />

コマンドラインには、HTTP ステータス行が印字される。

``` text
127.0.0.1 - - [29/Aug/2024 19:01:27] "GET / HTTP/1.1" 200 -
```

これによりサーバーが `GET` リクエストに応答していることがわかる。 Web ブラウザの開発者ツールを開くと、「ネットワーク」タブで HTTP リクエストと HTTP レスポンスの全体を確認することができる。

Web サーバーを止めるには、コマンドラインで <kbd>Ctrl+Break</kbd> キーを入力する。

実は `http.server` モジュールは、`simpleserver.py` のようなコードを書かなくても、`http.server.SimpleHTTPRequestHandler` を使用する `http.server.HTTPServer` インスタンスを作成し実行するコマンドラインインターフェースを備えている。

``` shell
python -m http.server [OPTIONS] [port]
```

`port` にポート番号を指定でき、省略した時は 8000 番ポートが使用される。`OPTIONS` に指定できる引数については、[公式ドキュメント](https://docs.python.org/ja/3.13/library/http.server.html#command-line-interface)を参照。

`http.server.SimpleHTTPRequestHandler` は、基本的に特定の HTML ファイルを配信することしかできない。現在では、HTTP サーバーとして求められる機能は、CSS ファイルや画像ファイルなどの配信、HTTPS や HTTP/2 による通信、負荷分散、キャッシュコントロールなど多岐にわたる。これらを全て `http.server.HTTPServer` をベースにインタプリタ言語である Python で実装しても、パフォーマンスに問題がある。[公式ドキュメント](https://docs.python.org/ja/3/library/http.server.html)でも『`http.server` は、本番環境では推奨されません。』と警告している。本番環境では、既存の高性能な HTTP サーバー（Nginx、Apache など）を使用すべきである。

Web アプリケーションサーバー
----------------------------

`RequestHandlerClass` が完成された HTML ファイルを配信するのではなく、動的にコンテンツを生成するような実装を考える。この場合でも、ゼロから作るのではなく、 `http.server.BaseHTTPRequestHandler` クラスを利用することができる。

`http.server.BaseHTTPRequestHandler` は、 `__init__()` メソッドが呼ばれると、`handle()` メソッドを呼び出し、リクエストとヘッダーを解析し、適切な `do_*()` メソッドへのディスパッチを行う。ここにディスパッチとは、処理すべきタスクを関数に割り当てることをいう。たとえば、 GET リクエストなら、 `do_GET()` メソッドが呼び出される。ただし、このクラスでは `do_*()` メソッドが一切定義されていないので、サブクラスにおいて `do_*()` メソッドを実装する必要がある。リクエストメソッドに対応する `do_*()` メソッドが定義されていない場合はエラーが発生する。

`http.server.SimpleHTTPRequestHandler` クラスは `do_GET()` メソッドを実装していた。いまは自作の `MyHTTPRequestHandler` クラスで `do_GET()` と `do_POST()` メソッドを実装する。

![](https://www.plantuml.com/plantuml/png/XP1DQiCm48NtEiKisuLzWYvCeTIRKbDxBuGrhKNruqZZG4XyzmX18E0ntesaRzxZKvD8HLgS5HP_2IX0D7yJWJOHHpB1Yz6gb225yHyfOJnZhDzLmXyyBPYeLLvRZ70L058QRqZAbzTycCf37bSkeQU8ocqo7kn4DDTFPUqmT1kOwVYwx90woC-F8PDbxBxxFBDjlN6pnQEKLJ1Vy7QhAk3416jRS5vEyDTbXN4yMDXJ4WrwlJXx1m00)

`http.server.BaseHTTPRequestHandler` クラスは、 `do_*()` メソッドの定義に便利な属性とメソッドをあらかじめ定義している。

| 属性 | 意味 |
|:---|:---|
| `BaseHTTPRequestHandler.path` | リクエストのパス。 URL にクエリが存在する場合、パスにはクエリが含まれる |
| `BaseHTTPRequestHandler.headers` | HTTP リクエストの各ヘッダーと値を保持するマッピングオブジェクト |
| `BaseHTTPRequestHandler.rfile` | `io.BufferedIOBase` 入力ストリーム。これはオプションの入力データの先頭から読み取る準備ができる |
| `BaseHTTPRequestHandler.wfile` | `io.BufferedIOBase` 出力ストリーム。これはクライアントに応答を書き戻す |

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `BaseHTTPRequestHandler.send_response(code, message=None)` | HTTP レスポンスを構成する HTTP ステータスラインを標準出力し、かつ、HTTP ス<br /><br />テータスラインと `Server` ヘッダーと `Date` ヘッダーを内部バッファに書き込む | `None` |
| `BaseHTTPRequestHandler.send_header(keyword, value)` | `keyword` で指定する HTTP ヘッダーを、その値を `value` として内部バッファに書<br /><br />き込む | `None` |
| `BaseHTTPRequestHandler.end_headers()` | HTTP レスポンス内の HTTP ヘッダーの終わりを示す空白行を内部バッファに書き<br /><br />込んだ上で、最終的にヘッダーを出力ストリームに送り、内部バッファをフラッシュ<br /><br />する | `None` |

`myappserver.py`:

``` python
from http.server import HTTPServer, BaseHTTPRequestHandler
from urllib.parse import unquote

class MyHTTPRequestHandler(BaseHTTPRequestHandler):
    def do_GET(self):
        self.send_response(200)
        self.send_header("Content-type", "text/html")
        self.end_headers()
        html_context = (
            '<html lang="ja">'
            '<head><meta charset="UTF-8"></head>'
            '<body><form method="POST" action="/">'
            '<input type="hidden" name="example" value="🍣">'
            '<input type="submit">'
            "</form></body>"
            "</html>"
        )
        self.wfile.write(html_context.encode())

    def do_POST(self):
        self.send_response(200)
        self.send_header("Content-Type", "text/plain; charset=utf-8")
        self.end_headers()
        data = self.rfile.read(int(self.headers["Content-Length"])).decode()
        data = unquote(data)  # パーセントデコード
        self.wfile.write(data.encode())

server_address = ("", 8000)
with HTTPServer(server_address, MyHTTPRequestHandler) as httpd:
    httpd.serve_forever()
```

作業ディレクトリに上記のファイルを作成する。コマンドラインで `python myappserver.py` を実行すると、 Web サーバーが立ち上がる。この状態で、 Web ブラウザを使って `http://127.0.0.1:8000/` という URLにアクセスすると、「送信」ボタンが表示される。ボタンを押すと、フォームデータとして `example=🍣` が送信される `POST` リクエストが発行される。 Web サーバーは、リクエストを受けると受信したフォームデータをそのまま返す。このため、 Web ブラウザでは `example=🍣` が表示される。

<img src="data:image/png;base64,
...
QmCC" />

今回は `http.server.BaseHTTPRequestHandler` クラスを継承したが、 `http.server.SimpleHTTPRequestHandler` クラスを継承してもよい。この場合、リクエストのパスによって機能を拡張するような処理を実装することが簡単になる。

今回のように動的に生成されるコンテンツに対して、完成された HTML ファイルや CSS ファイル、スクリプトファイル、画像ファイルなどは**静的ファイル**（static file）と呼ばれる。 Web サーバーは、静的ファイルを配信するだけでなく動的なコンテンツの生成に対応することによって、さまざまなサービスを提供することができるようになる。

現代では、ソフトウェアは Web サーバーを利用したサービスとして提供され、**Web アプリケーション**や **Software as a Service**（**SaaS**）と呼ばれる。 `myappserver.py` のように Web アプリケーションの開発に特化した Web サーバーのことを（静的ファイルを配信するための HTTP サーバーと区別する目的で） **Web アプリケーションサーバー**と呼ぶことがある。

WSGI
----

### WSGI の基本仕様

本番環境で動作するような Web アプリケーションを Python で開発しようとする場合、 `http.server.HTTPServer` の `RequestHandlerClass` に代わる仕様を決める必要がある。そうでなければ、ある本番環境で動作する Web アプリケーションが別の本番環境では動作しないということになってしまう。

そこで、 [PEP 3333](https://peps.python.org/pep-3333/) により、Web サーバーと Python で書かれた Web アプリケーションが通信するための共通インタフェースとして、**WSGI**（ウィズギー; Web Server Gateway Interface）が標準化された。これによって、 WSGI に対応した Web アプリケーションは、 WSGI に対応した任意の Web サーバー上で運用できるようになった。

WSGI に対応する Web アプリケーションに求められる条件は、「2 つの位置引数を受け入れる呼び出し可能オブジェクトであること」だけある。したがって、次の簡単な関数も WSGI の仕様に準拠した立派なアプリケーションである。

``` python
def application(environ, start_response):
    start_response('200 OK', [('Content-type', 'text/plain')])
    yield b"Hello world!\n"
```

第 1 引数には、環境変数として与えられる辞書が渡される。主なキーは以下の通り。

| 環境変数 | 意味 |
|:---|:---|
| `SERVER_NAME` | サーバー名 |
| `SERVER_PORT` | サーバーのポート番号（たとえば `80`） |
| `SERVER_PROTOCOL` | クライアントがリクエストの送信に使用したプロトコルのバージョン（たいてい `'HTTP/1.1'`） |
| `SCRIPT_NAME` | アプリケーションに対応するリクエスト URL のパスの最初の部分。アプリケーションがサーバーのルート `/` に対応する場合、これは空の文<br /><br />字列になる可能性がある |
| `PATH_INFO` | リクエスト URL のパスの中で、`SCRIPT_NAME` 該当部分より後ろの部分。これは空の文字列になる可能性がある |
| `QUERY_STRING` | リクエスト URL のうち `?` の後ろに続く部分。空か不在の可能性がある |
| `REQUEST_METHOD`| HTTP リクエストメソッド |
| `CONTENT_LENGTH` | HTTP リクエスト内の `Content-Length` フィールドの内容。空か不在の可能性がある |
| `CONTENT_TYPE` | HTTP リクエスト内の `Content-Type` フィールドの内容。空か不在の可能性がある |
| `HTTP_*` | HTTP リクエストに含まれる HTTP ヘッダーフィールドの各種フィールドに対応する。たとえば、`HTTP_HOST` は `Host` フィールドに対応する |
| `wsgi.version` | WSGI のバージョンを表すタプル `(1, 0)` |
| `wsgi.url_scheme` | URL のスキーム `http` または `https` |
| `wsgi.input` | HTTP リクエストボディのバイト列を読み取ることができる入力ストリーム |

第 2 引数には、2 つの必須引数と 1 つのオプション引数を持つコールバック関数が渡される。コールバック関数は、Web サーバーにおいて HTTP レスポンスヘッダーフィールドを保存するためのエンドポイントとして利用される。上記のコードでは、第 2 引数 `start_response` が以下のような引数を受け付けることになる。

``` python
start_resnpose(status, response_headers, exc_info=None)
```

| 引数 | 意味 |
|:---|:---|
| `status` | （必須引数）HTTP ステータスコードとリーズンフレーズからなる文字列（`'200 OK'`、`'404 Not Found'` など） |
| `response_headers` | （必須引数）HTTP ヘッダーフィールド名と値の 2 要素タプルからなるリスト |
| `exc_info()` | （オプション引数）`sys.exc_info()` が返す 3 要素タプル |

WSGI 対応 Web サーバーは、アプリケーションにおいてコールバック関数が適切な引数を伴って呼び出され、かつ、アプリケーションが、適切なサイズに分割されたレスポンスボディ（バイト列）を返すイテラブルを返すことを期待する。

Web サーバーとアプリケーションの間では、以下のようなやり取りが行われる:

  1. クライアントからのリクエストを受けた Web サーバーは、アプリケーションオブジェクトを呼び出す。その際に、リクエストなどの情報を環境変数として第 1 引数に渡す。また、コールバック関数を第 2 引数に渡す。
  2. アプリケーションがコールバック関数を呼び出すことによって、 Web サーバーは HTTP レスポンスヘッダーフィールドを保存する。さらに、アプリケーションは Web サーバーにイテラブルを返す。
  3. Web サーバーはアプリケーションから制御が返されると、保存した HTTP レスポンスヘッダーフィールドとイテラブルの最初の反復から HTTP レスポンスを作成してクライアントに送信する。もしイテラブルが複数のバイト列を返すようであれば、レスポンスボディをストリーミング形式でクライアントに逐次送る。

![](https://www.plantuml.com/plantuml/png/SoWkIImgAStDuKfCJyqhKL1IG0BXmUwU2e-RjZvktlEuUGAGH983gamiLYWlXkb3qR7eSVFqnysh7ZTj16bjslwuUVdZvit8-X8X-e1grpzkNlcuQGbOq-R7pHrSvSKAkdPO91Ii53n2GW8K7ZUlUToqyt7pNA0maC65gHAalxmOeNuLI4dJ2g1YcFaW1ObSvOb05bGfE1pYYrAB2_BpYbDHf416DxU1x6YUzhXvDPT3QbuACCG0)

実際の運用では、 WSGI サーバーが Web サーバーとして求められる全ての機能(HTTPS 通信など)をサポートしているわけではないので、既存の高性能な HTTP サーバーをプロキシとして WSGI サーバーの前段に置くことが多い。このようなプロキシを**リバースプロキシ**(reverse proxy)と呼ぶ。とくに、 WSGI サーバーは動的なコンテンツの生成に特化していて静的ファイルの配信には最適化されていないので、静的ファイルの配信はリバースプロキシに処理させるほうがパフォーマンスが上がる。

![](https://www.plantuml.com/plantuml/png/SoWkIImgAStDuKehIinDLL3ooSmhhL1utBJrSVEUnyqJWAIIGZLNOdfgNec2aW8I20zszqHHXoG7L9bOh51U3DE7ecFHu-RfZvjNFMxQ2DBQjFrny_B7pPkHzIN2z06L0Sq6Azi9rhNsSHCGRO_9Y_PBpyohq55mB4XCpaYD1UihAk2gXUW4shB-SVFAnqrBmDPjVjpSmLLUh62ZQmTKf62bu14I4i05P844C9L25GI3jEZgse7SKvu8OHK3vS0X5b-KMboGdrUSYY85CgMv3MHSyxJ7pQkv75BpKe1X2m00)

WSGI サーバーの例:

| サーバー | 開発言語 |
|:---|:---|
| Gunicorn | Python |
| uWSGI | C（Python C 拡張モジュール） |
| mod_wsgi | C（Apache モジュール）|

簡単な WSGI サーバーは、Python 標準ライブラリの `wsgiref` パッケージに含まれる `wsgiref.simple_server` モジュールでもサポートされている。`wsgiref.simple_server` は、`http.server` の上に構築されている。`wsgiref.simple_server.WSGIServer` は、 `http.server.HTTPServer` クラスを継承し WSGI に対応するように機能拡張したクラスである。 WSGI に対応する `RequestHandlerClass` として `wsgiref.simple_server.WSGIRequestHandler` も提供されているが、これは `http.server.BaseHTTPRequestHandler` クラスのサブクラスである。

WSGI サーバーの作成は、次の関数を呼び出して行うと簡単である。

``` python
wsgiref.simple_server.make_server(host, port, app, server_class=WSGIServer, handler_class=WSGIRequestHandler)
```

この関数は、 `host` と `port` 上で待機し、 `app` への接続を受け付ける WSGI サーバーを作成して返す。 `app` 引数には WSGI アプリケーションを渡す必要がある。

使用例 `wsgiapp.py`:

``` python
from wsgiref.simple_server import make_server

def application(environ, start_response):
    start_response("200 OK", [("Content-type", "text/plain")])
    yield b"Hello world!\n"

with make_server("", 8000, application) as httpd:
    print("Serving HTTP on port 8000...")
    httpd.serve_forever()
```

コマンドラインで `python wsgiapp.py` を実行すると、 WSGI サーバーが立ち上がる。この状態で、Web ブラウザを使って `http://127.0.0.1:8000/` という URLにアクセスすると、 `Hello world!` と表示される。サーバーを止めるには、コマンドラインで <kbd>Ctrl+Break</kbd> キーを入力する。



### リダイレクト

URL にアクセスしたときに自動的に別の URL に転送させる仕組みを**リダイレクト**という。

HTTP ステータスコードの `303 See Other` は別の URL に転送されることを表す。この場合、Web ブラウザはヘッダーフィールドに含まれる `Location` の URL を `GET` メソッドで表示する。`POST` メソッドでフォームの内容を送信した後には `GET` メソッドで処理結果を表示する URL へリダイレクトすることがよく行われる。

リダイレクト先を指定する `Location` フィールドの値はリクエスト URL と同じであってもよく、その場合、`303 See Other` でページがリロードされる。

リクエスト URL は、アプリケーションオブジェクト呼び出し時に第 1 引数として渡された環境変数から再構築することができる。PEP 3333 の [URL Reconstruction セクション](https://peps.python.org/pep-3333/#url-reconstruction)にコードがある。`wsgiref.util` モジュールでは、このコードを使った関数が提供されている:

``` python
wsgiref.util.request_uri(environ, include_query=True)
```

この関数は、リクエスト URL 全体（オプションでクエリ文字列を含む）を返す。第 1 引数には、アプリケーションオブジェクト呼び出し時に第 1 引数として渡された環境変数を渡す必要がある。

以下では、WSGI 対応の簡単な日記アプリケーションの例を示す。`python simplediary.py` を実行して WSGI サーバーを立ち上げた状態で `http://127.0.0.1:8000/` にアクセスすると、フォームが表示される。「記録」ボタンを押すと、`POST` リクエストが送信され、日記が追加されたページが再読み込みされる。

<img src="data:image/png;base64,
...
TkSuQmCC" />

`simplediary.py`:

``` python
from dataclasses import dataclass, field
from datetime import datetime
from string import Template
import html
from urllib.parse import parse_qs
from wsgiref.util import request_uri
from wsgiref.simple_server import make_server


HTML_TEMPLATE = """
<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8">
  <title>簡易日記</title>
 </head>
 <body>
  $entries
  <hr />
  <form action="/" method="post">
   <input type="text" name="diary" />
   <button>記録</button>
  </form>
 </body>
</html>
"""


@dataclass
class Entry:
    diary: str = ""
    entry_at: datetime = field(default_factory=datetime.now)


class SimpleDiary:
    entries: list[Entry]

    def __init__(self):
        self.entries = []

    def __call__(self, environ, start_response):
        match environ["REQUEST_METHOD"]:
            case "GET":
                return self.list_entries(environ, start_response)
            case "POST":
                return self.add_entry(environ, start_response)
            case _:
                # GET, POST 以外のメソッドの場合は 405 を返す
                start_response(
                    "405 Method Not Allowed", [("Content-type", "text/plain")]
                )
                return [b"Method Not Allowed"]

    def list_entries(self, environ, start_response):
        if len(self.entries) > 0:
            s = ""
            for entry in self.entries:
                s += f"<p>●[{entry.entry_at:%Y-%m-%d %H:%M:%S}] {entry.diary}</p>"
            context = {"entries": s}
        else:
            context = {"entries": "<p>日記はまだありません</p>"}
        message = Template(HTML_TEMPLATE).substitute(context)

        start_response("200 OK", [("Content-type", "text/html")])
        yield message.encode("utf-8")

    def add_entry(self, environ, start_response):
        # POST データを取得
        content_length = int(environ.get("CONTENT_LENGTH", "0"))
        input = environ["wsgi.input"].read(content_length)
        input = parse_qs(input).get(b"diary")
        if input:
            input = input[0].decode("utf-8")
            # 入力を全てエスケープして保存（XSS 攻撃対策）
            input = html.escape(input)
            self.entries.append(Entry(input))
        start_response(
            "303 See Other",
            [("Content-type", "text/plain"), ("Location", request_uri(environ))],
        )
        return []


if __name__ == "__main__":
    with make_server("", 8000, SimpleDiary()) as httpd:
        httpd.serve_forever()
```

### HTTP ヘッダーインジェクション

リクエスト URL のパラメータ（クエリ文字列）に `path` キーがあれば、そのパスにリダイレクトする WSGI アプリケーションを考える。例えば、Web ブラウザの URL 欄に `http://127.0.0.1:8000/?path=/foo` と入力したとき、`http://127.0.0.1:8000/foo` にリダイレクトする。次のようなコードを書いたとする。

``` python
from urllib.parse import urlparse, parse_qs
from wsgiref.util import request_uri
from wsgiref.simple_server import make_server

def application(environ, start_response):
    url = request_uri(environ)
    qs = urlparse(url).query
    query = parse_qs(qs)
    path = query.get("path", [""])[0]
    if path:
        start_response(
            "303 See Other",
            [("Content-type", "text/plain"), ("Location", path)],
        )
        return []
    start_response("200 OK", [("Content-type", "text/plain")])
    yield b"Hello world!\n"

with make_server("", 8000, application) as httpd:
    httpd.serve_forever()
```

12 行目で `start_response(..., [..., ("Location", path)])` と書いて、ユーザーの入力値を含める処理を実装していることに注目する。

では、この WSGI サーバーを立ち上げ、Web ブラウザの URL 欄に `http://127.0.0.1:8000/?path=/foo%0d%0aSet-Cookie: sessionId=12345678` を入力する。 `http://127.0.0.1:8000/foo` にリダイレクトされるときの HTTP レスポンスのヘッダーフィールドは次のようになる。

``` text
Content-Length: 0
Content-Type: text/plain
Location: /foo
Set-Cookie: sessionId=12345678
```

`Set-Cookie` はコードが明示的に追加したヘッダーではない。URL 欄に入力した `%0d%0aSet-Cookie: sessionId=12345678` から来ている。 `%0d%0a` は改行 `\r\n` を URL エンコードした値である。 HTTP では、改行がヘッダーフィールドの区切りという意味を持つ。このため、 `%0d%0a` 以降の文字列は 1 つのヘッダーフィールドとして扱われる。

このように、 HTTP レスポンスヘッダーフィールドの生成で、ユーザーの入力値を含める処理を実装していると、攻撃者が改行や空行を使って任意のヘッダーフィールドやコンテンツを追加したり、複数のレスポンスを作り出すような攻撃を仕掛けることに利用できてしまう。このような攻撃を **HTTP ヘッダーインジェクション**（HTTP header injection）攻撃という。

HTTP ヘッダーインジェクション攻撃の罠を踏んだ利用者の Web ブラウザでは、偽のページが表示されたり、スクリプトが実行されたり、任意の Cookie を保存させられたりする可能性がある。

対策としては、ユーザーの入力値をレスポンスのヘッダーフィールドに含めないことである。もし実装要件としてユーザーの入力値をレスポンスのヘッダーフィールドに含める必要がある場合は、改行コード以降の文字を削除する、あるいは、改行が含まれていたらレスポンス生成の処理を中止する必要がある。

---

HTTP ヘッダーインジェクション攻撃に似た攻撃として、**メールヘッダーインジェクション**攻撃もある。これは、Web アプリケーションがアンケートや問い合わせなどのためにメールを送信する機能を持つ場合に、フォームの宛先（`To` ヘッダー）や件名（`Subject` ヘッダー）などの入力値をメールヘッダーに含める処理を実装していると、攻撃者が改行を使って `Bcc` ヘッダーを追加する攻撃である。

`Bcc` ヘッダーは SMTP サーバーで削除され、同じ内容のメッセージが `Bcc` ヘッダーで指定した宛先に送信される。迷惑メールの送信に悪用される可能性がある。

対策は、HTTP ヘッダーインジェクションの場合と同様である。

### ミドルウェア

**ミドルウェア**（middleware）とは、Web サーバーと Web アプリケーションの両方のインターフェースを持つオブジェクトのことである。そのため、Web サーバー側からは Web アプリケーションのように見え、Web アプリケーション側からは Web サーバーのように見える。Web サーバーと Web アプリケーションの間にミドルウェアの処理が入ることによって、Web サーバーや Web アプリケーションの機能を拡張することができる。また、下図のように、ミドルウェアを層状に追加することができる。各ミドルウェアはそれぞれ Web サーバーとしても Web アプリケーションとしても振る舞うので、「サーバーからのアプリケーション呼び出し」が連鎖する。このようなミドルウェアの連鎖を**ミドルウェアスタック**と呼ぶ。ミドルウェアスタックにより機能拡張を柔軟に行える。

![](https://www.plantuml.com/plantuml/png/bP8zIi1048NxFSL2wXIcj93SmZgQ9GRNZSGLIJlJQ6Td9OA8yKSG4P60jtaNSLOCWew1D2aoNzvyyqXv5Ai-7XzuKpKkH0jw1j-3Ra3Nu5VmgSrys-p5flEr2trN3yxxxkGRttOxe3Vm1_XSdZRR7QemH_4L-0py2Be5ZHAuORF3Dkp_Eo_c-K-sI5o07q0lQGczWqVHcfrwvu9HbFDywu_QffMSnD8TQ1BA_2RaAqtQQ8VeRA6mHMANZZ4BipLxrlXh3QbzAEMc8f--AhnCRsWofGkr_4AV)

WSGI もミドルウェアの考え方を提供できる。WSGI ミドルウェアは、サーバー側とアプリケーション側の WSGI インタフェースを実装するものである。具体的には、WSGI の条件を満たす呼び出し可能オブジェクトがさらに WSGI アプリケーションを呼び出す。

例えば、以下のコードでは `Middleware` のインスタンスは何もしないが、立派な WSGI ミドルウェアである。

``` python
from wsgiref.simple_server import make_server

def application(environ, start_response):
    start_response("200 OK", [("Content-type", "text/plain")])
    yield b"Hello world!\n"

class Middleware:
    def __init__(self, app):
        self.app = app

    def __call__(self, environ, start_response):
        return self.app(environ, start_response)

middleware = Middleware(application)

with make_server("", 8000, middleware) as httpd:
    print("Serving HTTP on port 8000...")
    httpd.serve_forever()
```

次のコードでは、`Middleware` のインスタンスは簡易なログイン管理を行う WSGI ミドルウェアである。最初に `http://127.0.0.1:8000/` にアクセスすると、パスが `/login` のページに転送されてパスワード入力フォームが表示される。文字列 `password` を入力してから「Login」ボタンを押すと `Hello world!` が表示される。それ以外の文字列入力では、 Web ブラウザは「ページが見つかりません」と表示する。このコードは、パスワードの保護など必要なセキュリティを一切考慮していないことに注意する。

<img src="data:image/png;base64,
...
RU5ErkJggg==" />

``` python
from urllib.parse import urljoin, parse_qs
from wsgiref.util import request_uri
from wsgiref.simple_server import make_server

def application(environ, start_response):
    start_response("200 OK", [("Content-type", "text/plain")])
    yield b"Hello world!\n"

class Middleware:
    def __init__(self, app):
        self.app = app
        self.islogin = False
        self.password = "password"

    def __call__(self, environ, start_response):
        match environ["PATH_INFO"]:
            case "/":
                if self.islogin:
                    return self.app(environ, start_response)
                else:
                    start_response(
                        "303 See Other",
                        [("Content-type", "text/plain"), ("Location", urljoin(request_uri(environ), "login"))]
                    )
                    return []
            case "/login":
                method = environ["REQUEST_METHOD"]
                if method == "GET":
                    return self.login_form(environ, start_response)
                elif method == "POST":
                    return self.verify_password(environ, start_response)
            case _:
                return self.app(environ, start_response)

    def login_form(self, environ, start_response):
        form = """
        <html><head><meta charset="utf-8"></head><body>
        <form action="/login" method="post">
            <input type="password" name="password" /> <button>Login</button>
        </form>
        </body></html>
        """
        start_response("200 OK", [("Content-type", "text/html")])
        yield form.encode("utf-8")

    def verify_password(self, environ, start_response):
        content_length = int(environ.get("CONTENT_LENGTH", "0"))
        input = environ["wsgi.input"].read(content_length)
        input = parse_qs(input).get(b"password")
        if input is not None and self.password == input[0].decode("utf-8"):
            self.islogin = True
            start_response(
                "303 See Other",
                [
                    ("Content-type", "text/plain"),
                    ("Location", urljoin(request_uri(environ), "..")),
                ],
            )
            return []
        else:
            start_response("404 Not Found", [("Content-type", "text/plain")])
            return []

middleware = Middleware(application)

with make_server("", 8000, middleware) as httpd:
    print("Serving HTTP on port 8000...")
    httpd.serve_forever()
```

### Web アプリケーションフレームワーク

現代では Web アプリケーションの開発に Web アプリケーションフレームワークが利用されているが、 Python で実装された Web アプリケーションフレームワークのほとんどが WSGI をサポートしている。このため、実際の Web アプリケーション開発において WSGI を意識する必要はない。

<img src="data:image/png;base64,
iVBORw0KGgoAAAANSUhEUgAAATIAAAB6CAYAAAA8ut9wAAAABGdBTUEAALGPC/xhBQAAIABJREFU
eNrtnXd8VfX5x9/nnLtv9iKETAhhjzAVGfJjiFbROrBVsVqt1lGrP1urVu2w1Vpta+vParW1jlat
WvdAEQRFEWQFENkhIXvdm9zcfcbvjxtCQnKzSBj6fb9eafGe/T3nfM7zPN/n+3wlwzAMBAKB4CTG
JJpAMJA0hcI0BIKohg6cuN9MCQmLopBqt2FTFHHjhJAJBKAZBmXNXnY3NuFRVbQjRcxopyLt/vt4
yJ0EmCSZZKuFsYkJJNms4iaeREjCtRT0NwZQ3NTMF3X1fRAlowupabO8nRBKcMRj3KPjdrIdgF1R
mDEolTS7TdxMIWSCbyruUIhVFTU0h8NIknRYWdpqkdRHC+yIbftPetsz2G5nZnoqVuFmCtdS8M1k
t6uRplAoIg9GFJ/ROPxDr7+kx+DTWxcI0BAMMdhhFzdUCJngm0h9MIRqGCdGaN/owrBric0dMvKM
Nh6nT9VxB0KkO+z9bwAKhJAJTnxUTUfVe6MwUhfKI3WxbW8l5vB2BgaS0ebfbZcaYAj1EkIm+Gaj
G6DpelTZMdqZRVHMpraq0l8+Zi/CwYosI8LHQsgE32Qh03U0o4fxL+Molw+Q92mcKK6xQAiZ4Pig
GbRxLaO5gf3jWrbvDO1H6ZF6ZcAJhJAJvn6updGJa2l0dC17ZHb1o2vZG+R+FkaBEDLBySZkEavs
SCtJapGyzgRCOnae5OEDHZnX1tZ41I+/RRYOh1n27nu89drrLfl4Bqlpg/j5L+/Bbj9500L8fj8v
vfAin378Set1xcXHc/9DD2Iy9U2ShJAJBsQii4ytPFKdjk4ZDMAkSaTYLOi6Tk0gdDjhti92mtHF
vyV6HewPBAJUlJfT3NyMruk9tugkSSIzM5OU1NT2p2cYuFwNrFr+IYrZhGEYFIwaia5pJ/fzoetU
VVaw+sMVyKZIwvGQ7Kyj6lwRQiYYINfS6JGQHEpzkI78rY11JLW87FkOO+OT48mOdbKt9CCbDlSS
Pii9UzHrtZAdKS6y1CvZ3btnD6+89DJFmzdTVlJKMBDosUlns9u57w8PMfMIITvUALIsI7f0ovb5
Wk80JAlJiVwXBkd9XULIBP3vtRlGq2vZJzFp+U2WIMZkItNppzA1kcwYB1ZFQQJshs6H69Yxf/op
pKYNQu6LRXbkaPW2S3S9xxZCaUkJt//kp+zc/mWfrArdMNANXTw4QsgEJ5xrqfftxTQMMMsSqTYL
I+NjyUuIZUiMA6sit1vPYbVQ2tTI51+sZ8qkyaT1yTIz+iBxHXnvnXfZtWOHyDsTQib4ellkETFr
pxVS1xpiALIskRfrYFxyPMPiY0mwWjDJnYuT02oh1qRQ62rgi41fMHnyVFJTB/Wb69VT1zIYDFJe
XobRiSttGAb0IB9N1w1EB+nReQBCyAQDYpF1iJFFeVFlSSLWZCI7xs6kQUnkxDqxmxTkbgTJYbWS
aLbgD6vUud2s37CeaVNOISklpV/ETDJ65lr6vF6aGps6rDt85Ei+feEFTJk6ldi42K6PJUkkJiX1
64utqiqqqrYIbOcJL7quo6oquq5jsViIjY3tdt+qquL3+/H7/XiamggGg5Ft4+JwOBzYbDbMZvMx
fd5UVRVCJhgg17IHFoZVlpicksCE1EQyYx29qsxqNplIc9g44PMhAQ1uNxs3bWDixEkkpaYdtZjJ
hwradlM2SNd1tCN6EZ0xMVxz/XUsOvNM5CNc4oHG4/FQvH8/u3fuYs+ePXgaGyMW3xEmc1gN09TY
SMmBA/iam1l61VVce/11XYpjaUkJWzZv5r233uHzT9bgrq1D13Rkk0JCajJTTz2VeQsXcMqMU8nO
yYkE8vvpmirKytvJsSzL5A/PR5ZlzGazEDLBsXAtjZZSE+1XspvNTB6URG5cTO+FRpbJiImhuK6h
tVezzlXPps0bmFA4maTk1B6IWRfBflnus7c3aHA62TnZx1zEGt1unn36GV558UUqD5ZHrl+iUyWW
pEibSbKMoiio4XDUXlHDMPj0kzX844kn+fzjTzAkUBQFZ1JC6zphVWXNqlWs+egjps2ayY0/vokp
U6ce9QdF13We/sdTvPXa64d/0zQuu/IK8vPzRYxMcKxdy46y4A6p7HF5GBLjwNzLr7dhQJLD0eF3
V6ObLZs3MqFwKolJyVHyzDpLIGuf8yEbRp8TYi0WK2bTsXevnnvmWf7+18dQVRWz1dILP1rq+KFp
09BbN2/hTw8+xK4dO1rzvjr9uLRY1Bs+W8vvPB4e//vfSU1LParrWvvpZ/z7mWfwNDa1iup5Sy5k
8bfPa/ehEEImGACLzGgdNN4VmmawrsZFukVmeEoSth7EVrzBEGW1tazbuZut5ZXInRy7wdXA1i0b
GDOukMSU1OgvaRcZsbIhnVRDlPbt2cPzzz6LqqrtBS4cRg+ryC1pKwCSonQpSG3x+f2sXLmSL4u2
orTZZsLkSUyfcSoxMTE0N3tZv3YtRRs3RWKFksS+3btZsXw537n0kj5fU0V5Oc89/XSriAGMGDOa
S5cuJemImKIQMsEAWGRtXMuuBA8o8wX49+5SrrFaGdrGVTkSfyjEluIS1mz7kpVf7WKby02SzYoi
Sx2sP0mSaHA1sL1oE6MnFJKQFMXNlKIaZGhHU/3iGOesGobByg9X4GnydLCmvrP0MubOn4/Nbms9
sZIDxTz1xJOUlZR2H59q8rB65cpW60fTNOYvOoPbfn4nqampmM1mwuEwi889l8cffZR3Xn8j0tkQ
Vtm+fTt+nx97H6rsBgNBlr37Hms+Wn3Y0rVa+cG11zJy1KgO6wshEwyMRab3PCO2UZIo9QUZekTH
nabrNPn8bC0p5dW169lUUgoYTMzO4qazFxEMhbjvjXeoaPZ2umd3k5svizYzesIk4hKTImLWboxl
G/UyWspdSJF/y/Q9JcLv8+NyuWh0u7tUcZPZhMPpPOo4ktfrpaS0FKNN7p7ZbObWO27n2xdegKON
Cy5JEhmD03nv7Xd6JGSBYIDq8srWcww1+1h65RVkZWW1/mYymcgbmsdZZ5/N8mXvE/T7wTBobm4m
GAz2WsgMw2DLpk089bcn2lnF5110AfPPWIjSSaeQEDJB/wuZbqD1IlNdM2BnnZsZg1OxKDIhTaPO
3UjR/mLe3FzEf3ftZVx8LIvHj2X6mFEUDs0l1mZjf3UNefFxUYXMMAxc7gZ2bN3EiLETiE9KOSxm
0WJlLZ0TikGfE1xL9u/nmsu/16Vhpus6V994Az/44bXthKYvNDU2UV9X1+58C0aP4pQW169DLEuW
kWSpp6qCQft7GfD7OxXfsePHcf2Pf4TeIqjJycl9SsWoqanhgd/+lvq6OmRZRtd0zlh8NtfdeCMW
S+exPyFkgv4XMgzaG2RRchiMw72Z+zw+9rmb0H1e1u/axcfbv2LVwXLykxK5b+H/MGvMKIakpOC0
WVv3NCQpifxBg/isvDL6JHKShNvtYte2IoaPnUBcUkprFY7Ond2IlaYdZXxM62Zkg65pJCQmdGpd
9N4NC+Dzthfz/IICsrKyjnrfVquVpOQUmpo8SJKEJcbBf55/gZzcXLKzs1sD/IeE63tXXtmu7Xtb
zeJQZYw9u3e3pm9kD83lkksvJTU1eseBEDLBALiWHA72RytZ3bYihgG1wTB/XbeVA19uodrVQF5S
IveeexZzRo9iUEI8ZpOpffnFlppnaclJKJKE2o315G5ys2t7ESPGFhKTkNiFO2e0xvkGMtgvyzI2
m61fcq1C4RDBYLDdvpOTk7H2wyTDDqeTwqlTKN6/H0mSUBSFT1atprq6muzsbLJyshk/YQIjRo4k
Pj4eq9WG2WLu03Xpus4X69bz8gsvoGt6a1xs6RVXMGnK5C5dcCFkgn4nEuwninB1/ruuG9TLFkaP
GMVVaQlMHzmc9ISETm0nTyBAcWUVb23YzCubiwj05EE2DBrdDez6cgv5o8cTm5jcRW8mHK28KLLc
heUHiknB4XD2i5D5vD6aGhvbCVlqaiqSdPT7jomJYd7CBaz44IPW3kNd09i5/Uu+2rYdXdNRQyEw
ICM7i7POO4e58+Yxfvx47D11maVIJ0Lx/v28/OKL1FbXIMsymqpy1pKLuGDJRd1adkLIBAPkWvbe
mtGA3JxcFo7Ow2Hu+GiGVJUtxSWs2lLEW9u/ojoYZOmkicTZrNy3YjXmbo4pSRJN7gb2flnE0NET
iE1Miipm+lHkkaUNTufGm39M/vCCqAouSRLp6YP7Rch0XUdTtXbCoPRTMq4sy0ydNo1rb7iBP/7u
ATRNa7WMJElCMSkopkgwv66ulqce+xtv/vdVLr/qKr7/g6ujxrQ6fJwam/jH355gxbL3W9tEU1Um
TZ6Mzdb9jO9CyAQD61r2kk11bqbUNzI1PbnVYAuGw+yrqubVz9ezYvtXaJrO9GG5LJk1gzGZmTT7
/WwuOciKvft7FKBvanKz76siho6ZSExcwmExaxPKO5rJRxKTkhgzZiwjRo/qsiZZv9UWk47Yl0Fr
wL0/iI2N5dKllzFy1Chee+UVDh48iNvtxtPYhNfjIdxmVIDZYqHR3cg/n3iSrOwsvnXOOT2zKn0+
3n/nXaQ2wm62Wnnu6ac5ZcapDBkyRAiZ4NgLmd5HFXAHVZYVV5Ab58RpkjlYU8uyzUX8a90GVFXj
7DEjmD9pIlPz83G0ZK/bLGaunjeHsoYGvqp3dZ/GZRg0uRrY/+UWckeOx5mQ1EbBWqzD3tTx6aAr
UmT4T0StBry97XY7MXGxUH5YhH0+X78ew2K1cuppM5h2ynRcDQ1UV1dTVVlFdXU1dXW1rPtsLUUb
NrY2WbPHw8oVK5k5ezbx8fE9emg0TWv9EElSpA337tzFIw//mdvuuL1DEqwQMsEAC5nRcaByL9jh
9vLyjn2olaW8W7SNHY1NXDJ+DN+eOomCrEwSHI52Fogiy0wrGM71C+fx6zffo9br7VbMJEnC43Zx
4KutZI0YR0xiGzE75FqeJO1ts9lwOJ3tXM3q6hrUcBhTZ+kPkkRfs3YVRSElNZWU1FTGjB0bcfmD
QRadeRZ/+dOfWLX8w1YxKt63j8qKip4JWQvJKSnIkkRdXV3rs/TB2+8wfvx4LlhyEVZr5x0Ysnjt
BAOB3pM/IxIX0w79f8u//brOh1VuXjlYS0ZSIs987xLuvOBcpo4oIDFKAqnFZOKsyYX87qLzGJ2S
jKWb2JMsSZhkmeYmN6W7tuNtbIxUam1zbofcthNeyKxWHG2STg3D4GBpKTU1NR0+MH6/nz27dnPw
wIEe7dvV4KLsYBnlZWWUl5Xjdrs7uO8Wq5WCEQXMOf10TG1im3U1NXiamnp8Hc6YGG67604efuxR
0jMyWn8PBoM8/sj/sbWoKGroQAiZYMBcy27/Dq1L298jOWiqAYNzh3PxGWcya/Qo4hyObmuU2SwW
Fkwczx++dwnXzJjGyJRkArJMCAgbBmHDwGm1sCg/j6umTuLSKYXYTSY87noOfFVEs9uFrhut53Ky
VHyNS0gg5YjSRV+sXcsH739AbU0NaljF09TEtq3buO/X93LjD66hsryi+4+RrvPPJ59kesFoZk2a
ymnjC3n5xf8QapPq0ZakpCRk+XBeWTAQRFV7NlGKLMssvuB8Zs85nYmTJnHDj2/CajlsfdXV1nL7
//6EsoMHhWspOJauZc+DzUZnSgj4VI3/7q/EJEvMzBqEtQfJlYosMz4nm9y0VM4/zUVxZRXbysqp
aGwiEAozdWguZxROIMZuIxAOoxkG//xiI97GBsp2bSVj+FgcCUnoJ9EcH06nk+zs7EjpoZbaaOFQ
mEce+gPbt22jYMQIamtq+ODd96gur0Ax9+y1lySJvPxh2GKd2GKcGIbBJ6tXs+isM8nKzm63rqZp
HDhQ3G7QelxCfMsYz+5JSErksssvJz4h4oaecdaZVFVX88iDD0UGvEsS1VWVPPHY49x0yy0dqmoI
IRP0v5B1yOzv636g3Bfk7ztKqHK5ObMgl0Sno0e9fXF2O3FD7AwbnM7pE8a1pFMYmBQFi8kUKf6o
6WQkJ6IhYQaam9yU7/mSzJETMMXFnzQxMkmSmDptKs84nTS3ceUCgQDvvfEm77UVerOpV/vNHz6c
uIQEQmoYSZLY/MUGfvOrX7Fg4RmMGz8Oh9NJo7uRNZ98wqsvv4zWRshGjBpFZmZmj260zW4nY0hG
O3E+e/E5bCsq4pOVH0V6kXWDt19/gwmFE1l83nntUjuEkAkGQsnozzmB3KrOs/ur2VRykCVj8hmd
nUmMzdaj8JUiyygt8bKQquJq9mJSFA7W1fHY+x+ybOcebG0sQa+7nrKdReSOmsDJVEh/YmEh8xYu
4L/Pv4ApSu6WrmnExsVx6uxZ7N29hwP79nW73+zsbM5YfDavv/QKiklB13XWfLSa1R+uRFfVlrQL
GcVsas3/MgwDi8XC/IULOszV2Z1wHnnsq6+5hqJNm2h0RQbgh4JBHvj1vaQNGsTsOXOEkAkGVMf6
lBDbJSYzm4Mamz/4hDPS4piQP4wRWZkMSUzE0kVtLV8oRIXLTWlNDXtLy9hQfACnxcKeunqKqms7
xsEkCW9jA1V7tqMWFnRb6vpEwWyxcNU111BVUclnH3+M0sYNNwwDNRRi3pmLWHrF9xheUMDaTz/j
F3f+vCXeFf1excXHc+XVV1NbU8Nnqz9uHVupKEqn40R1XcfpcPDjn/2UhWec0SPruav+4cLJk7j9
rru54+ZboCXJ1+v18cC9vyE9PZ2CESOEkAkGMEbWYXbxaNMoHUrYktr8LnW6nWKxoaZm8tyeHby0
aSt5SQnkpaWSO3gQuampJDqdKLKEPxSmqrGR0tpaDtbUUVbfQLm7iZpAoMfT1Hk9jaheDwZGl0ON
JEnCGRNDZm5Ou3hPfwwGB7CYTQzJPTw4OzEpqV3SaFvyhuZxz72/4ul/PMXGDRtwNTSgazpZOdlc
evlSZp9+OnFxcciyzLwF81n27nvs3bsHWZYjaQ1S59dXUFDAb353P0//4yk2bdxIQ309Pq8PXdda
x/2bzGbi4uIYNnw4l11+OYWTJ0VNlZAAm81Odl5u5FoMg8Tk5E7XlWWZufPm8v0bruP9995rFUZN
13nlpZe5/kc3kpCQgGSIyfgE/cz9H29hRX1TH1UwmmIcXq6GgtQV78RddTASO2mZELjtl12WJBRJ
6rMxZTObefQ7F7JgSmGXvaWaqlJeXt6uJ89kMpOeMbhHQ2u6+yDUVFfjcrlaEmwlzBYzObm5XQ5t
0nWdyvJy9u/fTygYYtyE8aQNGtRhvYb6eurr65Ekifj4eFLT0ro9p0a3m5IDB6iuriEcCqHrOrIs
43Q6yczOIjcvr1sR13Wdmupqmj2HC0HKSqSmWTQLzu1yU1dX226khCRJpKalERcfL4RM0P/ct3oz
H9Y1dSlGUZfRs9CUGgzQULKbxqoyjAGYpbunQiY4MRB5ZIJjGzwzulnWw8+qyWojKaeAmJT0/huz
KDhpETEywYAI1rEw8xWrjeRhY5BkhaaaMhDOhRAygaA/Da9+67U0unY7FbOVxNwCADw15QPiZgqE
kAm+kUpm9K9xZHS9ULHYSMgpQFfDNDdUC8tMCJlA0A9I0jFPJVWsNpKGjQFFobm24ujFTJJ6PkGH
QAiZ4OvHoTphx9owki02EnJGABLe2oqjcjNjzSYcdrvosTxJEL2Wgn5nalYaVllu1xF5rP4Uq534
nALsSalHVdSwICGBISkp4mYKIRN8U5k+OJmzs9MIHadYlclqJ3HoGJwpg+nt+CIDsJjMnDN9Crlp
QsiEayn4xmJWZC4ZnYOqarxRXkdY14/5F1Ox2ojPiYzD89VV9qi2WNgwiHU4+NUZ8zhn+lSEU3ny
IDL7BQOCAfhCKl9U1vNmcQXFrub+H0jeg7PQgn68xV+hNbm7HJxssViYnD+MS2fNYGRmBtY+zJAt
EEIm+JqL2vGtiNPzg4tRAsK1FAg6F4fW/zmuZyD4pgmZgYEaVtlbUszqzz+luqaagM+PoQvjTdCN
ZMgSjpgYhmRkMPfUmWRlDMGkiO+l4BgLWSgcZt+B/fzh73/lrRXvE9RUVEMXydKCHhs/MmCSZWxm
K0vPvZDrl15FZsYQTP1Uo0sg6PDYtY2RBUMh3vnwfX79599TVFaM3SQCnoKjwx8OMnvURH75v7cz
c/qpmMUzJRhIITMMg3dXLuemX95OeWODyGgW9BuaYTA2I4fHf/cwUyZMFAF1wcAJWXFpCedc+R32
1VWJ0Kig39ENg8n5I3n7ny+SGJ8gGkTQr8iHrLHn//sSJfU1QsQEA/OgSRI7DuzlpTdfE40hGBgh
K60oZ8v2rWiilpNgAAlqGlu2bMbtaRKNIeh/IausrmRH6T7RGoIBZ1dpMWUV5aIhBP0vZMFAkLpm
8ZUUDDwNnkZ8fp9oCEH/C5mh6/jUkGgNwYATCIfQVE00hKBfMcGxGwunY3DelNnkZud0qLonSRLV
9XW8+slyQnrkQY+xWFk0dRYZg9I7XX/TV9v4dOfW4zAYWdBXDOM4D7sUfH2F7FgR1DWu/d73mTv7
9E6Xb9u9k5VffEqNLzJxZ0ZyGj+69nqmTijsdP0nX3iWL/bsIKCGxZ0UCL7pruWxRO9ivKau6+0s
L8OIuL1R1zcMMdGEQCAQFWIFAoEQMoFAIBBCJhAIBELIBALBN54TuuKdgYGqaaiq2qHeuizLaJrI
RxIIBCe4kFU31PHUs//kgw/e7yBkEhJffFnUmnMmEAiEkJ2QeEIBnl+9TNwlgUBw7IVM1XXCagjU
AGhBwADZDLJCb/O6DbpIBZekHpUdCusaqqaCGgQ9BLoKkgkUCygWzCYLJlnu23XqGmhh0EJgaJH9
I0f2LZvBZMWmmPq1mGDkuGrLcVUw1DY+t6n1T5YVzLLS4yKZYV1DDQch5AHVDzpgMoHJAWY7ZpO1
T+0kEJx0QmZWFKaOnsD0CYWMyBtKSkIyiizj8XnZfWA/jpiYHu9LkWWS4hKwWq0dxUwCT3MzjV5P
1O0lwGmzM3HkGMYOH8mwrFySExJxOhz4AwHqXA0crCrn8y0b2bBjW4/KGElEhkfF2ByMHJrP0Kwc
cjIyyRyUjsPuINYZg6ZpNHqaqHO72L77Kz7bsoGymqqjHkolIRFrtzNqWAGjhg4nM30waUnJOOwO
JElC13W8fj+uJjeVtTUcrCxnT0kxB6sqCKnqoc9Cp8iywv9MnMqsKdMZWzCS9JQ0bBYrde4Gqutq
2V28n1XrPmXznh2oIjYp+DoKmVlRmDVqInNnn863F51NVsYQJElq/aPFDjMMHVnq+Rc9Ly2Dx37z
ENMmTurkxZP527+f4a6H7yegHbZIEuxOJg0dybix45h5ygxmTp2Ow2Y/fC5trDjDMDAMA90w2Llv
D089/yxvrVhGZXNjG4sQRqZmMDZ/JBmZmYwZPZoJo8ZSkDcMk6K07DMiMu323WJJ6oaBoets+nIr
L7/5Gu+vXsH++uoeXb9mGGTGJzFlxFjGjB3L7FNPY/KY8distvbX0tbiMlrEyjBaz8Hn97Nj324+
27ieux5/GCkcGdKlYTAtdwTz58xlybnnMyw7F7nFgmu9by37Mwydu/T/pWjnl7z0xqusWvMx2ypL
RCFOwckvZJphEBMTw2+u/jEL5y9gWHZuN1v0zi2RkDCbTJijzPqsmNrPyjMyPYvrL7+KebPnkpeV
3atjjR85mt/cfjejh4/k9j/9tlUcA1qYay69kssu/i6xDmev2+jQGZ5SOIXCMeNZdPp8fvLAL9lV
diCqqBtAAJ2bzrmYhXPnMXXSZNKSUvp8n+LNZk4tnILb5SLQWI3dkURYgpsWf5dLLryYyeMnRr8H
rSIpgwJTxk1k4uixrN+8kT/+7VHe/Hy1cDcFJ7eQjUgfwm9/ejcLZs/FZrUe94vJTBvM6TNn91rE
DhHjcHLJ+Rfx4dqPefWTD7HICug6Qwal47TZj/r8rBYL82efzmOOBzntukuwhzt30ZJj4njwtntY
NGceCQkJ/WL1NHub+eOTj2KxJ6BIMjdeeCk/u+FmkhOTev/QKCZmTJlO9pBMtF/cwVtrV0XaSiA4
TvT5U5rijOXOG27l7PlnnBAi1urQHWUcyulwcMWF30Xz1g7IGUqSROG48fzfdbcROCJ1xAAm5RXw
yv/9g4vOPo/EfhIxgDeWL6Noz1eEDZ3z5yzk7ltu65OItftwDB7CX355PzNHTRCllAQnn5D5Ax5u
/t61LFl83kkztZfH20xVXQ0HKyuoqquJVNqIIjQTRo0BIzhg52K32Zg8cSIj0jLa/T45O58Hfv5r
pk2Y1K+T2da7XXz00Uo84SDjMvO486ZbiXV23eliGAaBUPdtkJE+mNuu+RGDYsXMSIKTyLXUDYPT
C0/j4nPP7zJwHwqHWLtpA7t27SLk94MBFoeNuTPnMHzosAG/ME3T2FW8j83bimioqaWyupr6RhfN
fh9Om53C8RP4wWVXYDJ1bAKzyQyDCjD83b/IwXCIFWs+pry8jKDXj6HrmKwWxk+YwIxJU6MKfWpq
GiMycympiwT+7YqZy5Z8l+mTJiN3EXNqaHTz2YZ1VJSXEwoEycrKZuHcedhttg7rupuaWPnZxxws
L2PFxrUEdJ1bLr+WvOycqPsPBAMs/3gVe3fvptnrJTExkZmnzWTi6LGdrq/IMjOmTWfRtFk8t/Jt
8UYJTg4hC2HwgwsvJS0lNeo6tfV1/OmJv/L6+29T5XETVCOB85AM7+cMHTAh03QNT3Mzby5fxr9f
/Q/79+2jvLGepmAA/YjUipXrP2XurDmMHDa8E6sMBjticPkD3R7T5XZz1/2/oqS+moCmYhgGJlkh
P3UwD9x9Lwtmnd65kCUlk5EyKHLehk5hwWguu+BiTErntyQcDvPWivd5/J9/Z195CQ3+ZprDIX58
0eXMnTUbO7ZOLLEGfvH73+LyeWjwe5mWM5xxY8ZgidJ5EgyFuP/Rh/mZw2gZAAAPxElEQVTnS/+m
PuDFMAwUWWbMa//h3jvuYcGsuZ1ulxAXz6Qpk3l59bJ2PcgCwQkrZOMzssnPH4bZZIridgZ48LFH
ePC5R7HbYtstU1X1qGNYXbF5/y4uvuUa9pYfwGqydUgEPZxIqlKOQUOjK1okC6fFiqsHp2oYBh5v
M/42VWpVXWNXdRkffbSS+TPndGqVOR0OnHZHROB9DdxwxdWt/93h4xEK8eIbr3LjA/cQDofbpY8E
g8GobWoymfAF/DT4vRHXdeS4Lj8i7320nEf+9Y92YqTpOlvLD/DUc88wevhIhqQP7nTb2aecRrzV
TsDnEW+V4MQXsrG5w8nMGBJ1+arP1/DU6y92ELFjgcvnpcHbjN1sb+cKBzUVTDLTR4xhfMEo8jKz
yc3IIj9naDQdw2IyH9W5GECDy0VTs4f42LhOXDIFs8mEAaRkDOWUwilR3dADZQe5+5HfEwqFOohz
V0F2m8Xaur4iyWRmZRIX0/l98fr9fLRqVVSLavmWdezZuyeqkOVn55KclEKVt+mkiZsKvsFCljko
g8Gpg6IuX//F+uNWQ/9Q1j1AmjOO8UNHMGbsWP5n1hwmjR5HjMMZSYyVJWRJRlbkrpWoH1zdrip0
SC1u5VmTT8PWSYzrkFC9vvxdSmvKsVsdR2wv4Qv6MaKUD7fbDlulZlkmOTV6OKC8qoK9pcVRl9f7
PBSXljIjHMJitnQUZkUhNS0VrXQfJiFkghNZyCRJwua0R/3i+oMBGurqj2tXvD/k5cozLmTxGWdx
2vRTSU1KPqFvQFjXGT1sOFaLJZrvyur1azGbbVFd266EsvVGywoJcfFR121saqLKVRd1uSzJ7C87
QDDUuZABpCQloxk6JlHmTnAiC5mMhMkc3eXyBwL4A4HjdjGGLPPA9XdwyYUXkzk44+S4A4bO4JRB
mLpwZRtcDZ0O/JakyGQu0UZQmkzm1o+OSZaJj4uLbj2qKoFwqEtR9Hi9Ua0/gFhHTIdOFYHghBMy
AwNdi/6gWs2W6JbFAKNIEteddwk/vPLqqHGg9i6fjiRJUTstjiVms6nLChVd5ZT5A4GoVpnFYmnd
r24YhELRhUpWlC7jggbgsNuR5Ojn6Q8GkCRhjQlOdCEzIByM/jI4HQ4Sk5KQJGlAeyc7I9kRy9Lv
XBI9mO3zUVldhc/rpby6kt0HisnNzubcBWce3zsgSTQ1N6PqGqYotyMvK4dPv9qKrZPUDIPuXEup
Rbx1Gj1NUdeNcTpJjUtkV0Vpp8t1XSc3Iytq6gZAVW0NJiFkguNAr546AwO324XX74u6TuGkSTiP
ssevt4R0jWmFkxmdX9Dp8oMV5fzxsUe47JYfMuHCOZx1zfnc/IdfsWvvnuN+AyQkahrq0FQ1is5J
XLDobAxfXd8s1RbhCRsaDQ2uqOtlZWQyPDsv6nKHzU5edg5WS+fD0ULhMBVlZShiALngRBcygH3l
pZRVVERdfubp8zjn9AUY/dBxJUm0L1ETNb4TZPyosUidvES6rvPAY3/hV0//la3Fe7Bb47HHpBJn
dfS44OCAupWyzOadXxKM4vZJksSMydM5f8GSPqU1mC1mDAzCuk5tVVXU9WKdTiZNnowzSiB//phJ
5ObkRt1+264duDxuUdZHcHII2cb9OzlYfjCq62iz2vjtz+7hzst/SGbKIPzN1fj9jfj9bvCU9crl
tJotKD0Zc6iFSU9Ji/oSbd29A+UEfcMUSea19R/h9Ua3clOSk7j31ju46uwL0awW/E3l+D2VhP31
JMTFdzmkyWQ2gxHp3dy5bzcHyg5GXXfJOefx/cVL8Aca8WthwrqGPxQgOT6B719yOcPzhkbdds3n
a2kKBsQbJTgu9DrSXef1sGLNaqZPnhp14HF6aho3X3sjSxZ/m+qaGkoqylEUmdTEZMaMGt1vJ98q
ipJMMMoAZ0mSuOL8i/nkro/AFHOoqw8/Opp+gvSweZtZtnoFV393aadWl4TE6IKR3P2/t3HtZVfi
drtRNZXkhCRi4mKJ6WGdtE93FLF3315yhmR2epxYZww/vfFmpk2eymvL36PaVcfkUeO4cNE5TBg7
Lup+y6sq+WLjF2IiGMHJI2RWWeF3//kn35q/iNOmTo/q7sQ6nYzIL2D40HxO1fVIsqosI8s9N40s
JnOXPXbhUCgS6lYs7CstwTCMDucjSRKXnb+EmdNOZdvunQQCARLi4snJGEJuH+uW9Tc2Wxx/e+4p
Fi88k0FdjGFNTUohNSklkqdnGF1aYq1i36YzoN7n4V+vv0zh+AlRS/ikJCVz7qJvcda8ha3HsFgs
Ue+zpml8sWkDyzZ+Jt4mwcnjWgLYDIkbfvkz9h7Y3/0BZBmzyYTJZEKRZaReRFFi7Y6oGe+H4l8G
BjaThTXrP4vaCWExWyjIG8YFZ3yLS8+9gG/Nnc/YEaN6bMkMNJIksavyIH9+8q80+7zdt6kk9UjE
Il632poZa5JknnnnRd798H3CavTB3SZFwW6zYbfbsVqtXcbmduzexc8feRB/KCjeJsHJJWSSJLG7
vJSf/PouNm7dErW219ES63Bi76IyaygcxjAi51NcXsrby0/eqeM0Q+f5t1/j2Zde6NekYlVV2308
7PZ4bv3Db3j7g2VdillP2LpzB/c8dB87Du4X4ysFJ5+QReI28OHmddx4x6089q+n8DQ3R6ZcOwoi
FtZhEuPiiYuNntwa8PlbZz7yqmEe/vtjfLJu7VG/oMeLmuZGfvf4n/nFH+6n3tXQo5nUu+s8UTvJ
+2v2+7jzwXt59uUXaPJ4ep3zFwqFWLN+LT+7926Wb/wMu2IWb5LguHJUae2GYVBUtp8ND97LUy+/
wPWXXMGMydOIiYnB4XDgsNmxWMwoR9RzD4XDBENB/H4/gWCQcChEMBRi/aYN+NVQq9WXlppGQmz0
YTVBn7/dS7j14H7OvvkqHrzhNubNnENcXBzxsXFRkzgNwyCsqgSCQQLBAKFQiHA4TF19HaFQ8NAo
dJo8HupcDZ26cw2N7qidBv5gkHq3q9Oxp7Is4+vE8qrzN/OX/zzDB5+u4vZrbmLy+InExMTgdDiw
WiK9uKFwGJ/fj8fjwdXoZtTwgk4t12AoFHXIUElDDVf/5nY++HQ1P/n+dWQOGUJKckrUkQ6GYdDg
dlHfUM9/3n6de57+KzYdkW4hOPmF7BBmWWZ7yV6uvucmiIln4ZgpjBk6nCFp6STExmG325FlGcMw
0FSVxuZmGhpdlFZVsKfsAEVlxbgrd4ESi90eES6rYiIzN3ol02AoiNfb3M6Ck4Cwz8+19/6EnCF5
nDl1FmPzC0hOTEJWFCRFRtc0DN3A0HRC4TBen5caVz0VtVVU1NVysLaK4voaVL8fCQmrYub5N17h
8y0bO3WfvH4fDYHO41obd27j9//3MLZOkkglSWLNps+jxsC+Kivhuz/9PslpOSwsnMbwrFySE5Kw
WSw0NTdTWlXOmq0bGZmXz6P3/r5TIauoqSLUhZVsN1l4edX7vPTxMpbMWMDcaTNISUnBardjs9kw
m82EQiG8zV6a3G4279jGi6uWUV1fjd1iF2+P4OslZBDJh7I7EkCHT7Zt5ONtGznUZyYhIUtSZKym
brTkuLYdQAP2mPY1ztJjEzlr7oKoxyurquRARVnnL6gtlpr6Op5e9hpgIEsyZsWESZYJ6xqaHkm9
MDDanYUURVRWfbmJj77c1KWb3anV46rlwPK3er1dpD0l7I4kfM0eXvtkxSG7qKU9I1sHNZXz5p4R
dbq8iuquhQxoHfb05tpVvLl2FU6LlRirDbvZitlkIqSG8fh9uAK+SK8wCBETfH2FrPOXtP1kuIeE
oTv8AQ9XXnwLQwalR48n1days6y4R+dgGAYhNUzoiGW96UGVjqod+qMt6XDGJpOZiaPG4ojSIVJe
VUmoh7OCH9qvLxTEFy0nT7wvghOUE25gXEBTuXTB+Sy98OKoPWGhcJi9+/ayq7bya3Mj/IFm1F72
/i4aN4Uxo0dHTcUoKSkhoIXFUy4QQnZMzUNZ4cJT/4fbf3QLg1LSoq7X2NTIc2+9gvlrYiM4Y2P5
9+8f4zvzzqIgZTAxlq7nCbUoJgqz87np2hsYGmV290aPhwPFxb0WR4FAuJZ9tUY0lZykVL5/7hKW
XnwJeZnRM+4Nw2D5xx+x/POPsDviT/obYAB1apD5M+aw5Ozz2LFnFxu3bKa6oori8lIq62vw+n3o
QKzNQd6QLPKHDmPhvPmMyMuPut+v9uxk7fZN4gkXCCE7FphlhR+cezHfWXw+E8aMw2HvOpC8oWgz
d//5918LEetwMxQT40eOYfzIMWiaRp2rgWafFzUcxiAy32ZCfDxJCYld2qI+v48VH6+iqKIEu6yI
p1wghGyg0QydOreL2NhYzGZzm165jpbYjj27uO8vD1Hlqv/a3xhFURiUksogUnu1na7rrF73GQ88
96QQMcE3huMeI9MNg9dWf8Bpl53HnQ/cy649u2lqbj83YiAYYEPRZn7481t5b8NnXVZF/Saj6Rqb
t23llnvvIqyKIL9AWGTHnHAoxEP/foLH33yJWy9YysxTZzBlQiG19XV8uGoldz7xZ5qbmzALK6NT
QuEwK9es5neP/JGSuqpepZYIBELI+hG72YYWCHDfv/7G0PfeYOqIsdQ3uvhs1zZCmvr1FTFNR1VV
NE1DluUeD8A2DANd1yk+WMLfn3+Wd1e8zx4hYgIhZCfISUkypQ01lK5d+bW/ARJgC4aZ970LuGjR
Ys6Zu4DBaemYLGYsZnNL+SOlxQ3XCYdVwuHImNCDFeW8+M7rvL78Xeo8TW1GKpzIiLCAYICErO0M
3YLjIGaSxIGqCu598mHu/fMdxGeN5dRho8nLyCQ1KRmHzYYkyfgDAWpd9RRXlLFm93aayraDczA2
s/WkscFkSdiLggESMsVkIt5mp76LeQ8FAy9mdosdLLmEPM18tHkdKzevw8CgtXiGFJkkWWoRBHtC
7kl3nbF2B5bjNPep4OuLDOBwOsmITxatcYJZLookYZJkzHLLnySjSNIJMftTX0lPTCGuixnPBYI+
C1ledg7jho0QrSEYcAry8skZkiUaQtD/QpYUn8D06dOxm0SlT8HAkWC1M2f2nC5nKxcI+ixkABcv
Pp8zT5mDXyRSCgYAvxrk4kXncubceaIxBAMnZHExsdx580/41uQZ+EXpF0G/iliIKxd+m1uv+xEm
xSQaRNDvSEabove6rrNr/17uf/ghln3+Mc1hMcWX4OjdyfPnn8XPbryFzMEZIs1HMPBCdohmr5dX
3nmDdevXsXXvTsobamkK+tFFLqOgGxRJIsHuIDtlEOOHj2LWrNksnr9IpFwIjr2QHcLtaeJAaQle
rxdNVUVOtqD7BwoJk9lEbGwsQ7NzcTocolEEx1fIBAKB4GTg/wHI3NCOzT7QrAAAAABJRU5ErkJg
gg==" />